# Análise Exploratória de Dados (EDA) - Dataset Principal de Plâncton (CPR)

Este notebook realiza a análise exploratória inicial do dataset Continuous Plankton Recorder (CPR) descarregado do GBIF.

**Formato:** Darwin Core Archive (DwC-A)
**Ficheiro Principal:** `occurrence.txt` (~3.2GB)

**Nota:** Devido ao tamanho do ficheiro `occurrence.txt`, a leitura e análise serão feitas usando `chunking` com Pandas para gerir o uso de memória.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

# Definir diretórios
project_dir = "/home/ubuntu/plankton_project"
data_dir = os.path.join(project_dir, "data")
plankton_data_dir = os.path.join(data_dir, "plankton_dwca")
report_dir = os.path.join(project_dir, "report")

# Criar diretório de report se não existir
os.makedirs(report_dir, exist_ok=True)

# Caminho para o ficheiro principal
occurrence_file = os.path.join(plankton_data_dir, "occurrence.txt")

print(f"Ficheiro de ocorrências: {occurrence_file}")
print("Bibliotecas importadas com sucesso!")


## 1. Leitura Inicial e Inspeção (Primeiro Chunk)

Vamos ler apenas o primeiro chunk do ficheiro `occurrence.txt` para inspecionar as colunas, tipos de dados e ter uma ideia inicial do conteúdo sem carregar o ficheiro inteiro.


In [ ]:
# Definir o tamanho do chunk (número de linhas)
chunk_size = 100000 # Ajustar conforme necessário
first_chunk = None # Inicializar a variável

try:
    print(f"A ler o primeiro chunk ({chunk_size} linhas) de {occurrence_file}...")
    # Ler o primeiro chunk, assumindo separador TAB (comum em DwC-A)
    # Adicionar error_bad_lines=False ou on_bad_lines="skip" se houver linhas mal formatadas
    # Adicionar low_memory=False se houver problemas com tipos mistos
    chunk_iter = pd.read_csv(occurrence_file, sep="\t", chunksize=chunk_size, low_memory=False, on_bad_lines="warn")
    first_chunk = next(chunk_iter)
    print("Primeiro chunk lido com sucesso.")

    # Inspeção inicial
    print("\nInformações do Primeiro Chunk:")
    first_chunk.info()

    print("\nPrimeiras 5 linhas:")
    # Usar display para melhor formatação no Jupyter
    from IPython.display import display
    display(first_chunk.head())

    print("\nColunas disponíveis:")
    print(list(first_chunk.columns))

except FileNotFoundError:
    print(f"Erro: Ficheiro não encontrado em {occurrence_file}")
except StopIteration:
    print("Erro: O ficheiro parece estar vazio ou o chunksize é maior que o ficheiro.")
except Exception as e:
    print(f"Erro ao ler o primeiro chunk: {e}")


## 2. Processamento Completo e Agregação (Todos os Chunks)

Agora, vamos iterar sobre todos os chunks do ficheiro `occurrence.txt` para:
- Selecionar colunas relevantes.
- Converter tipos de dados (datas, coordenadas).
- Realizar limpeza básica.
- Agregar estatísticas gerais (contagem total, intervalo temporal, espécies, etc.).


In [ ]:
# Colunas de interesse (ajustar conforme necessário)
use_cols = [
    'gbifID', 'eventDate', 'year', 'month', 'day',
    'decimalLatitude', 'decimalLongitude',
    'scientificName', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus',
    'basisOfRecord', 'occurrenceStatus', 'individualCount'
]

# Tipos de dados esperados (para otimização e consistência)
dtypes = {
    'gbifID': 'int64',
    'year': 'Int64', # Usar tipo Int64 que suporta NA
    'month': 'Int64',
    'day': 'Int64',
    'decimalLatitude': 'float64',
    'decimalLongitude': 'float64',
    'scientificName': 'str',
    'kingdom': 'str',
    'phylum': 'str',
    'class': 'str',
    'order': 'str',
    'family': 'str',
    'genus': 'str',
    'basisOfRecord': 'str',
    'occurrenceStatus': 'str',
    'individualCount': 'float64' # Pode ser float devido a NA ou contagens não inteiras
}

# Variáveis para agregação
total_records = 0
all_species = set()
min_date, max_date = pd.Timestamp.max, pd.Timestamp.min
min_lat, max_lat = 90.0, -90.0
min_lon, max_lon = 180.0, -180.0
species_counts = pd.Series(dtype='int64')
records_per_year = pd.Series(dtype='int64')

# Lista para guardar amostras de dados para visualização (se necessário)
data_samples = []
sample_fraction = 0.01 # Guardar 1% dos dados para visualização

print(f"Iniciando processamento completo de {occurrence_file} em chunks de {chunk_size} linhas...")

try:
    chunk_iter = pd.read_csv(
        occurrence_file,
        sep='\t',
        chunksize=chunk_size,
        usecols=lambda c: c in use_cols, # Ler apenas colunas de interesse
        dtype={k: dtypes.get(k, 'object') for k in use_cols}, # Aplicar dtypes, object para os não especificados
        low_memory=False,
        on_bad_lines='warn' # Avisar sobre linhas problemáticas
    )

    for i, chunk in enumerate(chunk_iter):
        print(f"Processando chunk {i+1}...")
        total_records += len(chunk)

        # Limpeza e Conversão de Tipos
        # Converter coordenadas para numérico, coercing erros para NaN
        chunk['decimalLatitude'] = pd.to_numeric(chunk['decimalLatitude'], errors='coerce')
        chunk['decimalLongitude'] = pd.to_numeric(chunk['decimalLongitude'], errors='coerce')

        # Tentar construir a data a partir de year, month, day
        # Usar Int64 para permitir NA em colunas de inteiros
        chunk[['year', 'month', 'day']] = chunk[['year', 'month', 'day']].astype('Int64')
        # Construir data apenas se year, month, day forem válidos
        valid_date_idx = chunk[['year', 'month', 'day']].notna().all(axis=1)
        # Usar um formato explícito e errors='coerce' para datas inválidas
        chunk.loc[valid_date_idx, 'parsedDate'] = pd.to_datetime(
            chunk.loc[valid_date_idx, ['year', 'month', 'day']],
            format='%Y%m%d', errors='coerce'
        )
        # Tentar preencher com eventDate se a construção falhar ou não for possível
        chunk['parsedDate'] = chunk['parsedDate'].fillna(pd.to_datetime(chunk['eventDate'], errors='coerce'))

        # Remover linhas sem data ou coordenadas válidas
        chunk.dropna(subset=['parsedDate', 'decimalLatitude', 'decimalLongitude'], inplace=True)

        if not chunk.empty:
            # Atualizar agregados
            min_date = min(min_date, chunk['parsedDate'].min())
            max_date = max(max_date, chunk['parsedDate'].max())
            min_lat = min(min_lat, chunk['decimalLatitude'].min())
            max_lat = max(max_lat, chunk['decimalLatitude'].max())
            min_lon = min(min_lon, chunk['decimalLongitude'].min())
            max_lon = max(max_lon, chunk['decimalLongitude'].max())

            all_species.update(chunk['scientificName'].dropna().unique())
            species_counts = species_counts.add(chunk['scientificName'].value_counts(), fill_value=0)
            records_per_year = records_per_year.add(chunk['parsedDate'].dt.year.value_counts(), fill_value=0)

            # Guardar uma amostra
            data_samples.append(chunk.sample(frac=sample_fraction))

    print("Processamento de chunks concluído.")

    # Concatenar amostras
    if data_samples:
        sample_df = pd.concat(data_samples, ignore_index=True)
        print(f"DataFrame de amostra criado com {len(sample_df)} registos.")
    else:
        sample_df = pd.DataFrame(columns=use_cols + ['parsedDate']) # Criar df vazio se não houver amostras
        print("Nenhuma amostra de dados foi criada.")

except Exception as e:
    print(f"Erro durante o processamento dos chunks: {e}")
    # Pode ser útil imprimir o estado das variáveis de agregação aqui



In [ ]:
print("\n--- Estatísticas Agregadas ---")
print(f"Total de registos processados (após limpeza inicial): {total_records}")
if min_date != pd.Timestamp.max:
    print(f"Intervalo Temporal: {min_date.strftime('%Y-%m-%d')} a {max_date.strftime('%Y-%m-%d')}")
else:
    print("Intervalo Temporal: Não foi possível determinar.")
print(f"Cobertura Geográfica (Lat): {min_lat:.4f} a {max_lat:.4f}")
print(f"Cobertura Geográfica (Lon): {min_lon:.4f} a {max_lon:.4f}")
print(f"Número de espécies únicas identificadas: {len(all_species)}")

print("\nTop 10 Espécies Mais Frequentes:")
print(species_counts.astype(int).nlargest(10))

print("\nNúmero de Registos por Ano (Top 10 anos):")
print(records_per_year.astype(int).nlargest(10))


## 3. Visualizações Preliminares

Vamos criar algumas visualizações com base nos dados agregados e na amostra recolhida.


In [ ]:
if 'species_counts' in locals() and not species_counts.empty:
    # Plot Top 10 Espécies
    plt.figure(figsize=(10, 6))
    top_species = species_counts.astype(int).nlargest(10)
    sns.barplot(x=top_species.values, y=top_species.index, palette='viridis')
    plt.title('Top 10 Espécies Mais Frequentes')
    plt.xlabel('Número de Ocorrências')
    plt.ylabel('Nome Científico')
    plt.tight_layout()
    save_path_species = os.path.join(report_dir, 'top_10_species.png')
    plt.savefig(save_path_species)
    print(f"Gráfico Top 10 Espécies salvo em: {save_path_species}")
    plt.close()
else:
    print("Dados de contagem de espécies não disponíveis para plotagem.")

if 'records_per_year' in locals() and not records_per_year.empty:
    # Plot Registos por Ano
    plt.figure(figsize=(12, 6))
    # Ordenar por ano para o gráfico de linha
    records_per_year_sorted = records_per_year.astype(int).sort_index()
    records_per_year_sorted.plot(kind='line', marker='o') # Usar gráfico de linha
    plt.title('Número Total de Registos por Ano')
    plt.xlabel('Ano')
    plt.ylabel('Número de Ocorrências')
    plt.grid(True)
    plt.tight_layout()
    save_path_years = os.path.join(report_dir, 'records_per_year.png')
    plt.savefig(save_path_years)
    print(f"Gráfico Registos por Ano salvo em: {save_path_years}")
    plt.close()
else:
    print("Dados de registos por ano não disponíveis para plotagem.")


In [ ]:
if 'sample_df' in locals() and not sample_df.empty:
    print(f"Gerando mapa interativo com {len(sample_df)} pontos da amostra...")
    try:
        fig = px.scatter_geo(
            sample_df,
            lat='decimalLatitude',
            lon='decimalLongitude',
            color='scientificName', # Colorir por espécie (pode ficar lento com muitas espécies)
            hover_name='scientificName',
            hover_data=['parsedDate'],
            projection='natural earth',
            title='Distribuição Geográfica das Ocorrências (Amostra)',
            # Limitar o número de categorias de cores se houver muitas espécies
            # color_discrete_sequence=px.colors.qualitative.Plotly[:10] # Exemplo
        )
        # Salvar como HTML interativo
        map_path = os.path.join(report_dir, 'plankton_distribution_map.html')
        fig.write_html(map_path)
        print(f"Mapa interativo salvo em: {map_path}")
    except Exception as e:
        print(f"Erro ao gerar o mapa interativo: {e}")
else:
    print("DataFrame de amostra não disponível para gerar mapa.")


## 4. Próximos Passos

- Análise mais aprofundada das distribuições geográficas e temporais.
- Limpeza de dados mais rigorosa (ex: validação de coordenadas, tratamento de outliers em `individualCount`).
- Análise do ficheiro `verbatim.txt` se contiver informações adicionais úteis.
- Download e EDA do dataset de Clorofila-a.
- Integração dos dados de plâncton com os dados ambientais (SST, Clorofila-a).
- Desenvolvimento do dashboard interativo.
